# Spam ham classifier
Consider a text classification problem. In this case, you will try to classify text as either spam or ham. To do this, you will apply concepts of Likelihood, prior, and posterior given a dataset comprising pairs of text and labels. There are two types of labels: 1 (spam) and 0 (ham). Your goal is to create a simple classifier that, when given, determines if the text is spam or ham.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
import re

## Data loading and cleaning

In [ ]:
data = pd.read_csv('spam_ham_dataset.csv')
data = data.dropna()
data = data.drop_duplicates()
df =data
nltk.download('stopwords')
stop = stopwords.words('english')


In [ ]:

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    # remove special characters
    text = [re.sub('\W+','', word) for word in text]
    
    return " ".join(text)

txt1 = df.text[0]
print(txt1)
# remove the filler words
txt1 = remove_stopwords(txt1)
print(txt1)
# apply the function to the entire dataset
df['text'] = df['text'].apply(remove_stopwords)
df.head()

Next we split the data into training and testing. We will derive the probabilities from the training data and then use them to predict the testing data.

In [ ]:
# lets split the data into training and testing
X = df.text
y = df.label_num
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### (1) 
Find the priors. What are the priors in this distribution? i.e find P (ham) and P (spam)

### (2) 
Find the likelihoods for each word. For each word in the dataset, find the likelihood that the word
is in spam and ham. This will represent the conditional probability P (w|spam) and P (w|ham) for
w where w ∈ V . V is the vocabulary of the dataset.

### (3) 
Define a function that, when given a text sequence, returns the probability of the text being in
spam. I.e., it returns P (spam|text). Note that this function calculates the likelihood using the
Bayes rule. Do the same for ham.

### (4) 
Perform inference, i.e., given a string of text, determine if it is ham or spam based on the poste-
rior probabilities calculated from the previous steps. Your function will determine the posterior
probability of your text being in ham and spam and classify it as being the larger of the two.

### (5)
Evaluate the data based on your test set and report the accuracy of your classifier. Your accuracy
must be greater than 85%.